# Clusteranalyse mit K-Means

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns    
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [ ]:
def Musterplot(ax, title, xlabel, ylabel, xticks, yticks):
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)

    ax.xaxis.set_tick_params(top='on', direction='in', width=1)
    ax.yaxis.set_tick_params(right='on', direction='in', width=1)
    
    ax.set_title(title,fontweight='bold',fontsize=14)
    
    ax.set_xlabel(xlabel,fontweight='bold',fontsize=12)
    ax.set_ylabel(ylabel,fontweight='bold',fontsize=12)
    
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

In [ ]:
df = pd.read_csv("Datensatz_Clusteranalyse.csv", sep=";")

In [ ]:
df2 = df[['Ankunftszeit','Abfahrtszeit','Ladeanteil','Leistung_max (kW)','Verbrauch (kWh)']]

In [ ]:
# Skalierung der Merkmale

scaler = StandardScaler()
X = scaler.fit_transform(df2)
X = pd.DataFrame(X, columns=df2.columns)


In [ ]:
## Ermittlung der Clusteranzahl mit Ellenbogen- und Silhoutten-Methode

inertia = []
silhouette = []

for k in range(2, 12):
    kmeans = KMeans(n_clusters=k, init = 'k-means++', random_state=42)
    labels = kmeans.fit_predict(X)
    labels2 = kmeans.fit(X)
    u = kmeans.inertia_
    inertia.append(u)
    score = silhouette_score(X, labels)
    silhouette.append(score)

fig, ax = plt.subplots()

xlabel = "k: Anzahl der Cluster"
ylabel = "J(k)"
title = "Ellenbogen-Methode"

yticks = np.arange(1000, 5000, step=500)
xticks = np.arange(2, 13, step=1)

# plt.annotate('Ellenbogen', xy=(5,2600), 
#              xytext=(6, 3500),fontweight='bold', arrowprops=dict(facecolor='red',shrink=0.1))

plt.grid()
ax.plot(range(2, 12), inertia, color='blue', marker='o')
Musterplot(ax, title, xlabel, ylabel, xticks, yticks)
plt.show()

fig2, ax2 = plt.subplots()

xlabel = "k: Anzahl der Cluster"
ylabel = "Silhouetten-Mittelwert"
title = "Silhouetten-Methode"

yticks = np.arange(0.26, 0.32, step=0.005)
xticks = np.arange(1, 13, step=1)
plt.grid()
ax2.plot(range(2, 12), silhouette, color='blue', marker='o')
Musterplot(ax2, title, xlabel, ylabel, xticks, yticks)

plt.show()

# fig.savefig("KMeans_Ellenbogen_5")
# fig2.savefig("KMeans_Silhoutte_5")

In [ ]:
## Initalisierung K-Means

kmeans = KMeans(n_clusters=5, init = 'k-means++', random_state=None)
y_kmeans = kmeans.fit_transform(X)
Labels = kmeans.labels_

In [ ]:
## Statistische Analyse der Cluster durch Mittelwert

df2['Labels'] = Labels
df2['Cluster'] = df2['Labels'].map({0:'Eins', 1:'Zwei',2:'Drei', 3:'Vier',4:'Fünf'})
df2['Cluster'] = df2['Cluster'].astype('category')
df2['Cluster'] = df2['Cluster'].cat.reorder_categories(['Eins','Zwei','Drei','Vier','Fünf'])

df3 = df2.groupby('Cluster').agg(
    { 
        'Labels':'count',
        'Ankunftszeit': 'mean',
        'Abfahrtszeit': 'mean',
        'Ladeanteil': 'mean',
        'Leistung_max (kW)':'mean',
        'Verbrauch (kWh)':'mean'
    }
).reset_index()

In [ ]:
df3

In [ ]:
## Plot für zweidimensionales Beispiel (Ankunfts- und Abfahrtszeit)

# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "Ankunftszeit in h"
# ylabel = "Abfahrtszeit in h "
# title = "Clusterergebnis mit K-Means (Ankunfts- und Abfahrtszeit)"
# yticks = np.arange(5, 25, step=1)
# xticks = np.arange(5, 25, step=1)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(df2['Ankunftszeit'], df2['Abfahrtszeit'], hue = df2['Labels'],palette="deep")
# # ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'], c = df2['Labels'])
# ax.scatter(kmeans.cluster_centers_[:,0] , kmeans.cluster_centers_[:,1] ,marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_A_A_Beispiel_KMeans")

In [ ]:
## Plot für Zweidimensionales Beispiel (max. Leistung und Gesamtsverbrauch)

# X['Labels'] = Labels

# fig, ax = plt.subplots(1,1,figsize=(7,5))

# xlabel = "max. Leistung (skaliert)"
# ylabel = "Verbrauch (skaliert)"
# title = "Clusterergebnis mit K-Means (max. Leistung und Verbrauch )"
# yticks = np.arange(-2, 5.1, step=0.4)
# xticks = np.arange(-2, 4, step=0.4)
# plt.grid()
# ax.set_axisbelow(True)
# sns.scatterplot(X['Leistung_max (kW)'], X['Verbrauch (kWh)'], hue = X['Labels'],palette="deep")
# # ax.scatter(df['Ankunftszeit'],df['Abfahrtszeit'], c = df2['Labels'])
# ax.scatter(kmeans.cluster_centers_[:,0] , kmeans.cluster_centers_[:,1] ,marker='x', s = 200, color = 'k')
# Musterplot(ax, title, xlabel, ylabel, xticks, yticks)

# plt.show()

# fig.savefig("Scatterplot_L_V_Beispiel_KMeans")

In [ ]:
## Naheliegenste Datenpunkte an den jeweiligen Zentroiden

df3 = df2.drop("Labels",axis =1)
Cluster_min = np.argmin(y_kmeans, axis=0)
Cluster_min = df3.to_numpy()[Cluster_min]
Cluster_min = pd.DataFrame(Cluster_min)
Cluster_min.columns = ['Ankunftszeit','Abfahrtszeit','Ladeanteil','Leistung_max (kW)','Verbrauch (kWh)','Cluster']
Cluster_min